# Entrando com as informações de acesso ao servidor

In [1]:
# plotar a frequencia de cursos por usuário
# EVG pegar a tabela tb_tematica_curso
# Pedir acesso a base da EVG (perfil de usuario no lugar da siape)
# Descrição do curso em variáveis

In [8]:
# -*- coding: utf-8 -*-

# Entrando com os pacotes necessários 

import pandas as pd # manipula os dados em DataFrame
import psycopg2 # conecta o banco sql ao python
import pandas.io.sql as sqlio # importa os dados do banco sql em formato Dataframe

# Colocando as informações de login no servidor

host = '10.224.9.157'
dbname= 'dw_consolidado' 
username = 'postgres' 
password = 'Enap@123' 

# Realizando a coneção com o banco de dados

conn = psycopg2.connect(host = host, dbname = dbname, user = username, password = password, port='5432')

# Importando as bases de dados 

In [9]:
# Puxando o banco de dados

sql = f"select desc_curso, modalidade_acao, categoria_acao, curso from tb_fato_matricula inner join dim_aluno using (dim_aluno_id) inner join dim_origem_dado using (dim_origem_dado_id) inner join dim_acao using (dim_acao_id) inner join dim_situacao using (dim_situacao_id) left join tb_fato_indicadores using (id_fato_matricula) left join dim_avaliacao using (dim_avaliacao_id) where (formato_acao = 'Curso') order by data_conclusao asc;"
df_acao = sqlio.read_sql_query(sql, conn) # Pegando as tabelas de interesse 

C:\Users\ferna\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [10]:
# Removendo duplicadas de curso com preferência para aqueles onde a descrição não é um campo nulo

df_acao = df_acao.drop_duplicates(subset=['curso'], keep='last') # Removendo duplicadas de cursos

# Modelo número 1: Cosine Similarity

## Realizando a limpeza dos dados

In [13]:
# Pegando apenas as colunas de interesse

df_acao = df_acao[["desc_curso", "modalidade_acao" , "categoria_acao","curso"]]

df_acao = df_acao.reset_index()
del df_acao['index']

# Seria interessante colocarmos um filtro de modalidade para o usuário 

'''
modalidade = ['Remota','A Distância']
df_acao_filtrado = df_acao[df_acao['modalidade_acao'].isin(modalidade)]
'''

df_acao_filtrado = df_acao

def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace('  ', ' ')) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace('  ', ' '))
        else:
            return ''
 
features = ["desc_curso", "modalidade_acao" , "categoria_acao","curso"]

for f in features:
    df_acao_filtrado[f] = df_acao_filtrado[f].apply(clean_data)
    
# Criando uma única coluna com todos os dados

import warnings
warnings.filterwarnings('ignore')

def create_soup(data):
    return data["curso"]+" "+ data["desc_curso"]+" "+ data["categoria_acao"]+" "+ data["modalidade_acao"]

df_acao_filtrado['soup'] = create_soup(df_acao_filtrado)

# Removendo caracteres repetidos

import re 

def cleaning_repeating_char(text):
    return re.sub(r'(.)1+', r'1', text)

df_acao_filtrado['soup'] = df_acao_filtrado['soup'].astype(str).apply(lambda x: cleaning_repeating_char(x))

# Removendo valores numéricos

def cleaning_numbers(data):
    return re.sub('[0-9]+', '', data)

df_acao_filtrado['soup'] = df_acao_filtrado['soup'].apply(lambda x: cleaning_numbers(x))

#  Removendo Emoticons 

df_acao_filtrado['soup'] = df_acao_filtrado['soup'].astype(str).apply(lambda x: x.encode('latin-1', 'ignore').decode('latin-1'))

# Removendo os stopwords

import nltk
nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words('portuguese')

df_acao_filtrado['soup'] = df_acao_filtrado['soup'].astype(str).apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

# Removendo acentuações

textlist = df_acao_filtrado['soup'].astype(str).values.tolist()

def traducao():
    string_origem = 'ãÃâÂáÁàÀêÊéÉèÈîÎíÍìÌïÏõÕôÔóÓòÒûÛúÚùÙüÜçÇñÑ'
    string_destino ='AAAAAAAAEEEEEEIIIIIIIIOOOOOOOOUUUUUUUUCCNN'
    string_anulada ='`´_-¿?.:ª°º,";()*/\n' + "\\" + "'" + '\t' 
    return string_origem.maketrans(string_origem, string_destino, string_anulada)

for i in range(0,len(textlist)):
    textlist[i] = textlist[i].translate(traducao()).lower()
    
df_acao_filtrado['soup'] = textlist

# Aplicando o stemming

st = nltk.PorterStemmer()

def stemming_on_text(data):
    text = [st.stem(word) for word in data]
    return data

df_acao_filtrado['soup']= df_acao_filtrado['soup'].apply(lambda x: stemming_on_text(x))

# Aplicando o Lemmatizer

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

lm = nltk.WordNetLemmatizer()

def lemmatizer_on_text(data):
    text = [lm.lemmatize(word) for word in data]
    return data

df_acao_filtrado['soup'] = df_acao_filtrado['soup'].apply(lambda x: lemmatizer_on_text(x))

# Serve para contar a frequência das palavras e alocar em uma matrix 2D

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

count_vectorizer = CountVectorizer()
count_matrix = count_vectorizer.fit_transform(df_acao_filtrado["soup"])

# Modalidade de cálculo das distâncias 

cosine_sim2 = cosine_similarity(count_matrix, count_matrix) 

df_acao_filtrado = df_acao_filtrado.reset_index()
indices = pd.Series(df_acao_filtrado.index, index=df_acao_filtrado['curso'])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ferna\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ferna\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Realizando a previsão dos cursos

In [14]:
pd.set_option('display.max_colwidth', None)

def get_recommendations(title, cosine_sim=cosine_sim2):
    
    # Pega o índice do curso em interesse
    idx = indices[title]

    # Compara os scores com o curso base
    
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Ordena os scores
    
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Separa os top 10 cursos (pula primeira linha pois o próprio filme é aquele mais correlacionado com ele mesmo)
    sim_scores = sim_scores[1:4]

    # Separa os indices dos cursos
    movie_indices = [i[0] for i in sim_scores]

    # Retorna os valores dos top 10 cursos calculados
    
    recomendacao = []
    
    recomendacao.append(df_acao_filtrado['curso'].iloc[movie_indices])
    
    return recomendacao


print("microeconomia")
print()
print(get_recommendations("microeconomia", cosine_sim2))


microeconomia

[254            macroeconomia
554              estatística
458    matemática financeira
Name: curso, dtype: object]


## Realizando a previsão dos cursos (base cheia)

In [15]:
### Sistema de recomendação automatico da base

recomendacao_coisine = []
curso_coisine = []

for i in range(0,len(df_acao)):
    try:
        recomendacao_coisine.append(str(get_recommendations(df_acao['curso'][i], cosine_sim2)).replace('[','').replace(']','').split('\n'))
        curso_coisine.append(df_acao['curso'][i])
    except:
        pass
  
### Transformando em Dataframe

Coisine_indicacao = pd.DataFrame(recomendacao_coisine ,columns=['Recomendacao 1','Recomendacao 2','Recomendacao 3','Recomendacao 4'])
Coisine_indicacao['Curso'] = curso_coisine      
del Coisine_indicacao['Recomendacao 4']


for i in range(0,len(Coisine_indicacao)):
    if '  ' in str(Coisine_indicacao['Recomendacao 1'][i]):
       Coisine_indicacao['Recomendacao 1'][i] = str(Coisine_indicacao['Recomendacao 1'][i]).split('  ')[-1] 
    
    
for i in range(0,len(Coisine_indicacao)):
    if '  ' in str(Coisine_indicacao['Recomendacao 2'][i]):
       Coisine_indicacao['Recomendacao 2'][i] = str(Coisine_indicacao['Recomendacao 2'][i]).split('  ')[-1]
    
    
for i in range(0,len(Coisine_indicacao)):
    if '  ' in str(Coisine_indicacao['Recomendacao 3'][i]):
       Coisine_indicacao['Recomendacao 3'][i] = str(Coisine_indicacao['Recomendacao 3'][i]).split('  ')[-1]
    
    
### Exportando em formato csv

Coisine_indicacao.to_csv("Coisine_indicacao.csv", index=False, encoding='utf-8-sig', sep = '|')

Coisine_indicacao

,Recomendacao 1,Recomendacao 2,Recomendacao 3,Curso
0,dw aduaneiro,contencioso aduaneiro e tributário no âmbito da receita federal,reuniões produtivas,fiscalização aduaneira: conceitos aplicados pela receita federal
1,"instrumentos de planejamento: ppa, ldo e loa","reorganização societária, blindagem patrimonial e responsabilidade para a receita federal",o arrolamento de bens para a proteção do crédito tributário no âmbito da receita federal,planejamento sucessório e tributário aplicado à receita federal
2,conceitos essenciais para avaliação socioeconômica de projetos,estatística,matemática financeira,oficina programa avaliação socioeconômica de projetos (asp)
3,transferências da união: visão geral,termo de compromisso: atos preparatórios,validação do relatório de melhoria da gestão das transferências da união,transferências especiais
4,operação de drones no âmbito da receita federal,noções introdutórias de licitação e contratos administrativos,siafi básico,atendimento pré-hospitalar em acidentes com armas de fogo aplicado à receita federal
...,...,...,...,...
559,noções introdutórias de licitação e contratos administrativos,siafi básico,formação de pregoeiros - teoria,curso básico de licitações - enfrentando (e vencendo) tabus
560,a construção dos livros didáticos do pnld,materiais didáticos do pnld para o novo ensino médio,pnld literário,pnld – programa nacional do livro e do material didático
561,liderança e gestão de equipes,noções básicas do trabalho remoto,estruturas de gestão pública,a liderança pública em tempos de crise
562,ações inovadoras da cgu,formação de conteudistas para cursos virtuais - módulo 1,formação de facilitadores de aprendizagem,solução pacífica de conflitos no âmbito da administração pública


# Modelo número 2: LDA

## Realizando a limpeza dos dados

In [16]:
# Importing modules
import pandas as pd
import numpy as np
import os
import re

# LDA Model
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
from pprint import pprint
from gensim.models import CoherenceModel
import spacy
from nltk.corpus import stopwords

# Import the wordcloud library
from wordcloud import WordCloud

# Visualize the topics
import pyLDAvis.gensim
import pickle 
import pyLDAvis

# Pegando apenas as colunas de interesse

features = ["desc_curso", "modalidade_acao" , "categoria_acao", "curso"]
df_acao = df_acao[features]

df_acao.drop_duplicates(subset=['curso'], keep='last')

df_acao = df_acao.reset_index()
del df_acao['index']

# Seria interessante colocarmos um filtro de modalidade para o usuário 

'''
modalidade = ['Remota','A Distância']
df_acao_filtrado = df_acao[df_acao['modalidade_acao'].isin(modalidade)]
'''
df_acao_filtrado = df_acao

# Criando uma única coluna com todos os dados

import warnings
warnings.filterwarnings('ignore')

def create_soup(data):
    return data["curso"]+" "+ data["desc_curso"]+" "+ data["categoria_acao"]+" "+ data["modalidade_acao"]

df_acao_filtrado['soup'] = create_soup(df_acao_filtrado)

# Tokenizando 

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
    
data_words = list(sent_to_words(df_acao_filtrado['soup']))
df_acao_filtrado['Tokens'] = data_words

# Removendo valores numéricos

for i in range(0,len(df_acao_filtrado)):
    
    df_acao_filtrado['Tokens'][i] = re.sub('[0-9]+', '', str(df_acao_filtrado['Tokens'][i]))
    
#  Removendo Emoticons 

df_acao_filtrado['Tokens'] = df_acao_filtrado['Tokens'].astype(str).apply(lambda x: x.encode('latin-1', 'ignore').decode('latin-1'))

# Removendo os stopwords

import nltk
nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words('portuguese')

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stopwords] for doc in texts]

# Removendo acentuações

teste = []
def traducao():
    string_origem = 'ãÃâÂáÁàÀêÊéÉèÈîÎíÍìÌïÏõÕôÔóÓòÒûÛúÚùÙüÜçÇñÑ'
    string_destino ='AAAAAAAAEEEEEEIIIIIIIIOOOOOOOOUUUUUUUUCCNN'
    string_anulada ='`´_-¿?.:ª°º";()*/\n][' + "\\" + "'" + '\t' 
    return string_origem.maketrans(string_origem, string_destino, string_anulada)

for i in range(0,len(df_acao_filtrado['Tokens'])):
    df_acao_filtrado['Tokens'][i] = str(df_acao_filtrado['Tokens'][i]).translate(traducao()).lower()

# Bigram

bigram = gensim.models.Phrases(df_acao_filtrado['Tokens'], min_count=5, threshold=10) 
bigram_mod = gensim.models.phrases.Phraser(bigram)

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

# Lemmatizer

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

import spacy 

spacy.cli.download("pt_core_news_sm")

# Removendo stopwords

data_words_nostops = remove_stopwords(df_acao_filtrado['Tokens'])

# Bigramas

data_words_bigrams = make_bigrams(data_words_nostops)

nlp = spacy.load("pt_core_news_sm", disable=['parser', 'ner'])

# Lematização

data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# Criando um dicionario

id2word = corpora.Dictionary(data_lemmatized)

id2word.filter_extremes(no_below=2, no_above=0.9)

# Criando o corpus

texts = data_lemmatized

# Pegando as frequencias

corpus = [id2word.doc2bow(text) for text in texts]

# Estruturando o modelo

lda_model = gensim.models.LdaMulticore(corpus=corpus,
 id2word=id2word,
 num_topics=15, 
 random_state=100,
 chunksize=100,
 passes=10,
 alpha=0.01,
 eta=0.9)

# Avaliando o score de coerência

coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

print('Score de Coerência: ', coherence_lda)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ferna\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
Score de Coerência:  0.3203319165818728


## Criando uma base de scores

In [17]:
# Removendo os stopwords

import nltk
nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words('portuguese')

df_acao_filtrado['soup'] = df_acao_filtrado['soup'].astype(str).apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

count_vectorizer = CountVectorizer()
sparse_matrix = count_vectorizer.fit_transform(df_acao_filtrado["soup"])


# Convertendo uma matriz sparsa em DataFrame Pandas

doc_term_matrix = sparse_matrix.todense()
matrix_df = pd.DataFrame(doc_term_matrix, 
                  columns=count_vectorizer.get_feature_names(), index=df_acao_filtrado.index)

similarity_scores = cosine_similarity(sparse_matrix, sparse_matrix) 

scores_df = pd.DataFrame(similarity_scores )
scores_df

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ferna\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,0,1,2,3,4,5,6,7,8,9,...,556,557,558,559,560,561,562,563,564,565
0,1.000000,0.128004,0.048002,0.078762,0.150643,0.308429,0.236286,0.111149,0.166759,0.104770,...,0.068079,0.071842,0.094234,0.088983,0.073817,0.124124,0.062062,0.066704,0.114309,0.074901
1,0.128004,1.000000,0.047619,0.046881,0.163028,0.161985,0.171896,0.091886,0.099258,0.207870,...,0.027015,0.071270,0.040064,0.039233,0.029292,0.092351,0.076960,0.033086,0.070874,0.018576
2,0.048002,0.047619,1.000000,0.046881,0.040757,0.053995,0.046881,0.055132,0.049629,0.044544,...,0.040522,0.106904,0.080128,0.029424,0.043937,0.046176,0.184703,0.049629,0.085049,0.167183
3,0.078762,0.046881,0.046881,1.000000,0.120375,0.070877,0.061538,0.090462,0.065146,0.043853,...,0.053192,0.087706,0.039443,0.067593,0.028837,0.166686,0.121226,0.032573,0.027910,0.036576
4,0.150643,0.163028,0.040757,0.120375,1.000000,0.154047,0.147125,0.173019,0.169910,0.088957,...,0.080926,0.106749,0.057151,0.083948,0.050141,0.223956,0.197608,0.056637,0.060661,0.031798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,0.124124,0.092351,0.046176,0.166686,0.223956,0.104717,0.060613,0.106921,0.112291,0.071989,...,0.078588,0.103664,0.155399,0.095109,0.071010,1.000000,0.164179,0.064167,0.109961,0.036026
562,0.062062,0.076960,0.184703,0.121226,0.197608,0.087264,0.060613,0.142562,0.112291,0.043193,...,0.065490,0.069109,0.090650,0.057065,0.085211,0.164179,1.000000,0.048125,0.151197,0.108077
563,0.066704,0.033086,0.049629,0.032573,0.056637,0.056274,0.048860,0.057459,0.068966,0.046424,...,0.056310,0.074278,0.069592,0.051111,0.091584,0.064167,0.048125,1.000000,0.132958,0.038720
564,0.114309,0.070874,0.085049,0.027910,0.060661,0.112509,0.111640,0.114878,0.088639,0.066296,...,0.072373,0.095467,0.107333,0.096347,0.078473,0.109961,0.151197,0.132958,1.000000,0.033177


## Realizando a previsão dos cursos

In [18]:
def Sort_Tuple(tup):  
    return(sorted(tup, key = lambda x: x[1], reverse = True))   

doc_num, topic_num, prob = [], [], []
print(lda_model.get_document_topics(corpus))

for n in range(len(df_acao)):
    get_document_topics = lda_model.get_document_topics(corpus[n])
    doc_num.append(n)
    sorted_doc_topics = Sort_Tuple(get_document_topics)
    topic_num.append(sorted_doc_topics[0][0])
    prob.append(sorted_doc_topics[0][1])
    
df_acao['Doc'] = doc_num
df_acao['Topic'] = topic_num
df_acao['Probability'] = prob

def recommend(title,scores_df, df):
    recommended = []
    
    # getting title's index 
    title = title.lower()
    df['curso'] = df['curso'].str.lower()
    index = df[df['curso']==title].index[0]
    
    top10_list = list(scores_df.iloc[index].sort_values(ascending = False).iloc[1:30].index)
    
    for each in top10_list:
        recommended.append(df.iloc[each].curso)
    
    recommended = list(dict.fromkeys(recommended))
    
    return recommended[1:4]

recommend("microeconomia",scores_df, df_acao)

['estatística',
 'matemática financeira',
 'conceitos essenciais para avaliação socioeconômica de projetos']

## Realizando a previsão dos cursos (base cheia)

In [19]:
### Sistema de recomendação automatico da base

recomendacao_LDA = []
curso_LDA = []

for i in range(0,len(df_acao)):
    try:
        if len(recommend(df_acao['curso'][i],scores_df, df_acao)) != 0:
            recomendacao_LDA.append(recommend(df_acao['curso'][i],scores_df, df_acao))
            curso_LDA.append(df_acao['curso'][i])
    except:
        pass
  
### Transformando em Dataframe

LDA_indicacao = pd.DataFrame(recomendacao_LDA ,columns=['Recomendacao 1','Recomendacao 2', 'Recomendacao 3'])
LDA_indicacao['Curso'] = curso_LDA      

### Exportando em formato csv

LDA_indicacao.to_csv("LDA_indicacao.csv", index=False, encoding='utf-8-sig', sep = '|')

LDA_indicacao

,Recomendacao 1,Recomendacao 2,Recomendacao 3,Curso
0,dw aduaneiro,reuniões produtivas,contabilidade com foco na gestão do orçamento público,fiscalização aduaneira: conceitos aplicados pela receita federal
1,"reorganização societária, blindagem patrimonial e responsabilidade para a receita federal",o arrolamento de bens para a proteção do crédito tributário no âmbito da receita federal,representação fiscal para fins penais no âmbito da receita federal,planejamento sucessório e tributário aplicado à receita federal
2,estatística,matemática financeira,"programa de gestão em projetos urbanos: curso iv - projetos no âmbito do programa de minha casa, minha vida",oficina programa avaliação socioeconômica de projetos (asp)
3,termo de compromisso: atos preparatórios,validação do relatório de melhoria da gestão das transferências da união,plano de melhoria da gestão das transferências da união,transferências especiais
4,noções introdutórias de licitação e contratos administrativos,siafi básico,direitos dos imigrantes e orientações para o atendimento,atendimento pré-hospitalar em acidentes com armas de fogo aplicado à receita federal
...,...,...,...,...
561,siafi básico,formação de pregoeiros - teoria,impactos da mudança do clima para a gestão municipal,curso básico de licitações - enfrentando (e vencendo) tabus
562,materiais didáticos do pnld para o novo ensino médio,pnld literário,a importância do pnld,pnld – programa nacional do livro e do material didático
563,noções básicas do trabalho remoto,estruturas de gestão pública,gestão em ouvidoria,a liderança pública em tempos de crise
564,formação de conteudistas para cursos virtuais - módulo 1,formação de facilitadores de aprendizagem,estruturas de gestão pública,solução pacífica de conflitos no âmbito da administração pública
